**This is example of how to trace model with jit and export it to the onnx**

In [1]:
from rl_games.torch_runner import Runner
import ray
import yaml
import torch
import matplotlib.pyplot as plt
import gym
from IPython import display
import numpy as np
import onnx
import onnxruntime as ort
%matplotlib inline

In [2]:
ray.init(object_store_memory=1024*1024*1000)

2022-03-30 17:10:01,407	INFO services.py:1274 -- View the Ray dashboard at http://127.0.0.1:8265


{'node_ip_address': '192.168.0.81',
 'raylet_ip_address': '192.168.0.81',
 'redis_address': '192.168.0.81:6379',
 'object_store_address': '/tmp/ray/session_2022-03-30_17-10-00_620352_30954/sockets/plasma_store',
 'raylet_socket_name': '/tmp/ray/session_2022-03-30_17-10-00_620352_30954/sockets/raylet',
 'webui_url': '127.0.0.1:8265',
 'session_dir': '/tmp/ray/session_2022-03-30_17-10-00_620352_30954',
 'metrics_export_port': 53872,
 'node_id': '2afdfc87648d495904201447645e81338eca9f3953661775c24e721e'}

In [3]:
config_name = 'rl_games/configs/ppo_cartpole.yaml'

In [4]:
with open(config_name, 'r') as stream:
    config = yaml.safe_load(stream)
    config['config']['full_experiment_name'] = 'cartpole_onnx'
runner = Runner()
runner.load(config)
runner.run({
    'train': True,
})

KeyError: 'config'

In [ ]:
agent = runner.create_player()
agent.restore('runs/cartpole_onnx/nn/cartpole_vel_info.pth')

import rl_games.algos_torch.flatten as flatten
inputs = {
    'obs' : torch.zeros((1,) + agent.obs_shape).to(agent.device),
    'rnn_states' : agent.states
}
with torch.no_grad():
    adapter = flatten.TracingAdapter(agent.model.a2c_network, inputs,allow_non_tensor=True)
    traced = torch.jit.trace(adapter, adapter.flattened_inputs,check_trace=False)
    flattened_outputs = traced(*adapter.flattened_inputs)
    print(flattened_outputs)
    
torch.onnx.export(traced, *adapter.flattened_inputs, "cartpole.onnx", verbose=True, input_names=['obs'], output_names=['logits', 'value'])

onnx_model = onnx.load("cartpole.onnx")

# Check that the model is well formed
onnx.checker.check_model(onnx_model)

In [ ]:
ort_model = ort.InferenceSession("cartpole.onnx")

outputs = ort_model.run(
    None,
    {"obs": np.zeros((1, 4)).astype(np.float32)},
)
print(outputs)

In [ ]:
is_done = False
env = agent.env
obs = env.reset()
#prev_screen = env.render(mode='rgb_array')
#plt.imshow(prev_screen)
total_reward = 0
num_steps = 0
while not is_done:
    outputs = ort_model.run(None, {"obs": np.expand_dims(obs, axis=0).astype(np.float32)},)
    action = np.argmax(outputs[0])
    obs, reward, done, info = env.step(action)
    total_reward += reward
    num_steps += 1
    is_done = done
    screen = env.render(mode='rgb_array')
    #plt.imshow(screen)
    #display.display(plt.gcf())    
    #display.clear_output(wait=True)
print(total_reward, num_steps)
#ipythondisplay.clear_output(wait=True)